In [14]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.optimizers import RMSprop
from keras.optimizers import SGD
import os
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [2]:
base_model = InceptionV3(weights="imagenet", include_top=False)# モデルロード

In [3]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [4]:
# inceptionの部分は学習させない
for layer in base_model.layers:
    layer.trainable = False

In [5]:
import cv2
from keras.datasets import cifar10
import numpy as np

In [6]:
class CIFAR10Dataset():

    def __init__(self):
        self.image_shape = (190, 190, 3)# inceptionのsizeに変更
        self.num_classes = 10# クラス数
        self.train_data_size = 5000
        self.test_data_size = 5000
    
    def upscale(self, x, data_size):
        data_upscaled = np.zeros((data_size,
                                  self.image_shape[0],
                                  self.image_shape[1],
                                  self.image_shape[2]))
        for i, img in enumerate(x):
            large_img = cv2.resize(img, dsize=(self.image_shape[0],
                                               self.image_shape[1]),)
            data_upscaled[i] = large_img
        return data_upscaled
    
    def get_batch(self):
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()# ロード
        # 画像を拡大
        x_train = self.upscale(x_train, x_train.shape[0])
        x_test = self.upscale(x_test, x_test.shape[0])
        x_train = x_train[:self.train_data_size]
        y_train = y_train[:self.train_data_size]
        x_test = x_test[:self.test_data_size]
        y_test = y_test[:self.test_data_size]
        
        x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
        y_train, y_test = [self.preprocess(d, label_data=True) for d in
                           [y_train, y_test]]

        return x_train, y_train, x_test, y_test

    def preprocess(self, data, label_data=False):
        if label_data:
            
            data = keras.utils.to_categorical(data, self.num_classes)
        else:# 正規化
            data = data.astype("float32")
            data /= 255  
            shape = (data.shape[0],) + self.image_shape  
            data = data.reshape(shape)

        return data

In [7]:
class Trainer():

    def __init__(self, model, loss, optimizer):
        self._target = model
        self._target.compile(
            loss=loss, optimizer=optimizer, metrics=["accuracy"]
            )
        self.verbose = 1
        logdir = "logdir_" + os.path.basename("__file__").replace('.py', '')
        self.log_dir = os.path.join(os.path.dirname("__file__"), logdir)
        self.model_file_name = "model_file.hdf5"

    def train(self, x_train, y_train, batch_size, epochs, validation_split):
        if os.path.exists(self.log_dir):
            import shutil
            shutil.rmtree(self.log_dir)
        os.mkdir(self.log_dir)

        datagen = ImageDataGenerator(
            featurewise_center=False, 
            samplewise_center=False,
            featurewise_std_normalization=False,  
            samplewise_std_normalization=False, 
            zca_whitening=False,  
            rotation_range=0, 
            width_shift_range=0.1,  
            height_shift_range=0.1, 
            horizontal_flip=True, 
            vertical_flip=False) 


        datagen.fit(x_train)

        indices = np.arange(x_train.shape[0])
        np.random.shuffle(indices)
        validation_size = int(x_train.shape[0] * validation_split)
        x_train, x_valid = \
            x_train[indices[:-validation_size], :], \
            x_train[indices[-validation_size:], :]
        y_train, y_valid = \
            y_train[indices[:-validation_size], :], \
            y_train[indices[-validation_size:], :]

        model_path = os.path.join(self.log_dir, self.model_file_name)
        self._target.fit_generator(
            datagen.flow(x_train, y_train, batch_size=batch_size),
            steps_per_epoch=x_train.shape[0] // batch_size,
            epochs=epochs,
            validation_data=(x_valid, y_valid),
            callbacks=[
                TensorBoard(log_dir=self.log_dir),
                ModelCheckpoint(model_path, save_best_only=True),
                EarlyStopping(),
            ],
            verbose=self.verbose,
            workers=4
        )

In [8]:
dataset = CIFAR10Dataset()

In [9]:
x_train, y_train, x_test, y_test = dataset.get_batch()

In [10]:
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=RMSprop())

In [11]:
# 数エポック学習
trainer.train(
    x_train, y_train, batch_size=26, epochs=8, validation_split=0.2
    )




Epoch 1/8
153/153 [==============================] - 12s 77ms/step - loss: 1.8090 - accuracy: 0.4540 - val_loss: 1.2672 - val_accuracy: 0.6520

Epoch 2/8
153/153 [==============================] - 6s 42ms/step - loss: 1.1427 - accuracy: 0.6122 - val_loss: 1.5570 - val_accuracy: 0.6540


In [15]:
model = load_model(os.path.join(trainer.log_dir, trainer.model_file_name))

In [16]:
# 250層以降を学習できるようにする
# inceptionの上位層部分のみを学習するイメージ
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

In [17]:
trainer = Trainer(model, loss="categorical_crossentropy",
                  optimizer=SGD(lr=0.001, momentum=0.9))

In [18]:
trainer.train(
    x_train, y_train, batch_size=26, epochs=8, validation_split=0.2
)

Epoch 1/8
153/153 [==============================] - 10s 64ms/step - loss: 0.8103 - accuracy: 0.7214 - val_loss: 1.0416 - val_accuracy: 0.7090
Epoch 2/8
153/153 [==============================] - 7s 44ms/step - loss: 0.6660 - accuracy: 0.7680 - val_loss: 0.9725 - val_accuracy: 0.7280
Epoch 3/8
153/153 [==============================] - 7s 45ms/step - loss: 0.5299 - accuracy: 0.8171 - val_loss: 0.9860 - val_accuracy: 0.7300


In [19]:
model = load_model(os.path.join(trainer.log_dir, trainer.model_file_name))
# show result
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.9504465588569642
Test accuracy: 0.7271999716758728
